In [ ]:
import NanobodyPaperPlotting
import matplotlib.pyplot as plt
import numpy as np
import warnings
from collections import defaultdict

colors_decimal = [[120/255, 120/255, 120/255], [249/255, 29/255, 0/255], [32/255, 25/255, 250/255]]
colors_decimal_darker = [[50/255, 50/255, 50/255], [162/255, 19/255, 0/255], [25/255, 19/255, 192/255]] # 2+?, 3, 2 "tint"s darker according to adobe color

expressionLims = (10**0.25, 10**3)
BDFPPositiveBDFPSSCLims = (10**-1, 10**3)
BDFPNegativeBDFPSSCLims = (10**-4, 10**1)

expressionLims_kde = (0.25, 3)
# BDFPPositiveBDFPSSCLims_kde = (-1, 3)
BDFPPositiveBDFPSSCLims_kde = (0, 3)
BDFPNegativeBDFPSSCLims_kde = (-4, 1)

AmFRETLims = (-0.2, 2)
AmFRETLims_sesA = (-0.2, 1.5)

figsize = (6.5, 1)
mpl_dpi=200
fontSize = 6 #changed from 7 
tickFontSize = 4
plt.rcParams.update({'font.size': fontSize})

#for population summary text
textX = 0.03
textYMax = 0.97

In [ ]:
def reformatAxs(axs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for i, row in enumerate(axs):
            for j, ax in enumerate(row):
                ax.xaxis.set_label_coords(0.5, -0.25)
                ax.yaxis.set_label_coords(-0.2, 0.5)
        
                ax.tick_params(axis='both', which='major', pad=1, width=0.5, length=2.5) #default length is 3.5
                # ax.minorticks_off()
                ax.tick_params(axis='both', which='minor', width=0.5, length=1.5)           #default length is 2
                ax.xaxis.set_ticklabels(ax.xaxis.get_ticklabels(), fontsize=tickFontSize)
                ax.yaxis.set_ticklabels(ax.yaxis.get_ticklabels(), fontsize=tickFontSize)
                
                for spine in ax.spines.values():
                    spine.set_linewidth(0.5)

def reformatAxs1D(axs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # for i, row in enumerate(axs):
        for j, ax in enumerate(axs):
            ax.xaxis.set_label_coords(0.5, -0.25)
            ax.yaxis.set_label_coords(-0.2, 0.5)
    
            ax.tick_params(axis='both', which='major', pad=1, width=0.5, length=2.5) #default length is 3.5
            # ax.minorticks_off()
            ax.tick_params(axis='both', which='minor', width=0.5, length=1.5)           #default length is 2
            ax.xaxis.set_ticklabels(ax.xaxis.get_ticklabels(), fontsize=tickFontSize)
            ax.yaxis.set_ticklabels(ax.yaxis.get_ticklabels(), fontsize=tickFontSize)
            
            for spine in ax.spines.values():
                spine.set_linewidth(0.5)

In [ ]:
#These hand gated files are no longer necessary
# ASCFiles = [
#     ("Control", ["data/ASC/gated/gate0/A11_x3912b.fcs",
#               "data/ASC/gated/gate1/A11_x3912b.fcs",
#                 "data/ASC/gated/gate2/A11_x3912b.fcs"]),
#     ("1x mEosNb", ["data/ASC/gated/gate0/G11.fcs",
#               "data/ASC/gated/gate1/G11.fcs",
#                   "data/ASC/gated/gate2/G11.fcs"]),
#     ("2x mEosNb", ["data/ASC/gated/gate0/H11.fcs",
#               "data/ASC/gated/gate1/H11.fcs",
#                   "data/ASC/gated/gate2/H11.fcs"]),
#     ("3x mEosNb", ["data/ASC/gated/gate0/A11.fcs",
#               "data/ASC/gated/gate1/A11.fcs",
#                   "data/ASC/gated/gate2/A11.fcs"]),
#     ("4x mEosNb", ["data/ASC/gated/gate0/B11.fcs",
#               "data/ASC/gated/gate1/B11.fcs",
#                   "data/ASC/gated/gate2/B11.fcs"]),
#     ("5x mEosNb", ["data/ASC/gated/gate0/C11.fcs",
#               "data/ASC/gated/gate1/C11.fcs",
#                   "data/ASC/gated/gate2/C11.fcs"]),
#     ("6x mEosNb", ["data/ASC/gated/gate0/D11.fcs",
#               "data/ASC/gated/gate1/D11.fcs",
#                   "data/ASC/gated/gate2/D11.fcs"])
# ]

ASCUngatedFiles = [
    # ("Control", ["data/ASC/complete/exported_plate1/A11.fcs"]), #x3912b is no longer the prefered control - unknown level of effect possible.
    ("Control", ["data/ASC/complete/exported_plate2/G11.fcs"]), #"Control" forces bdfp-
    ("1x mEosNb", ["data/ASC/complete/exported_plate2/G11.fcs"]),
    ("2x mEosNb", ["data/ASC/complete/exported_plate2/H11.fcs"]),
    ("3x mEosNb", ["data/ASC/complete/exported_plate3/A11.fcs"]),
    ("4x mEosNb", ["data/ASC/complete/exported_plate3/B11.fcs"]),
    ("5x mEosNb", ["data/ASC/complete/exported_plate3/C11.fcs"]),
    ("6x mEosNb", ["data/ASC/complete/exported_plate3/D11.fcs"]),
]

sesAUngatedFiles = [
    # ("Control", ["data/SesA/exported_plate4/A1.fcs"]),
    ("Control", ["data/SesA/exported_plate5/G1.fcs"]), #"Control" forces bdfp-
    ("1x mEosNb", ["data/SesA/exported_plate5/G1.fcs"]),
    ("2x mEosNb", ["data/SesA/exported_plate5/H1.fcs"]),
    ("3x mEosNb", ["data/SesA/exported_plate6/A1.fcs"]),
    ("4x mEosNb", ["data/SesA/exported_plate6/B1.fcs"]),
    ("5x mEosNb", ["data/SesA/exported_plate6/C1.fcs"]),
    ("6x mEosNb", ["data/SesA/exported_plate6/D1.fcs"]),
]

In [ ]:
ASC_standardPoints_2pops = { #this should be 2!!
    1: [
    [0.9, 0.35],
    [2000, 0.35],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}


ASC_2pops_perPlotBoundaries = {
    "Control": ASC_standardPoints_2pops,
    "1x mEosNb": ASC_standardPoints_2pops,
    "2x mEosNb": ASC_standardPoints_2pops,
    "3x mEosNb": ASC_standardPoints_2pops,
    "4x mEosNb": ASC_standardPoints_2pops,
    "5x mEosNb": ASC_standardPoints_2pops,
    "6x mEosNb": ASC_standardPoints_2pops   
}

In [ ]:
# ASC_Nb1x_points_3pops = {
#     1: ASC_standardPoints_2pops[1],

#     2: [
#     # [3, 0.72],
#     # [2000, 0.72],
#     [0.9, 0.75],
#     [2000, 0.75],
#     [2000, 1.5],
#     [0.9, 1.5],
#     ]
# }

# ASC_Nb2x_points_3pops = {
#     1: ASC_standardPoints_2pops[1],

#     2: [
#     [0.9, 0.88],
#     [2000, 0.88],
#     [2000, 1.5],
#     [0.9, 1.5],
#     ]
# }

# ASC_Nb3x_points_3pops = {
#     1: ASC_standardPoints_2pops[1],

#     2: [
#     [0.9, 0.96],
#     [2000, 0.96],
#     [2000, 1.5],
#     [0.9, 1.5],
#     ]
# }

ASC_Nb1x_points_3pops = {
    2: ASC_standardPoints_2pops[1],

    1: [
    # [3, 0.72],
    # [2000, 0.72],
    [0.9, 0.75],
    [2000, 0.75],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

ASC_Nb2x_points_3pops = {
    2: ASC_standardPoints_2pops[1],

    1: [
    [0.9, 0.88],
    [2000, 0.88],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

ASC_Nb3x_points_3pops = {
    2: ASC_standardPoints_2pops[1],

    1: [
    [0.9, 0.96],
    [2000, 0.96],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

ASC_Nb5x_points_3pops = {
    2: ASC_standardPoints_2pops[1],
}

# ASC_3pops_perPlotBoundaries = {
#     "Control": ASC_standardPoints_2pops,  #this only has 2 pops
#     "1x mEosNb": ASC_Nb1x_points_3pops,
#     "2x mEosNb": ASC_Nb2x_points_3pops,
#     "3x mEosNb": ASC_Nb3x_points_3pops,
#     "4x mEosNb": ASC_Nb3x_points_3pops,
#     "5x mEosNb": ASC_standardPoints_2pops, #this only has 2 pops
#     "6x mEosNb": ASC_standardPoints_2pops   #this only has 2 pops
# }

ASC_3pops_perPlotBoundaries = {
    "Control": ASC_standardPoints_2pops,  #this only has 2 pops
    "1x mEosNb": ASC_Nb1x_points_3pops,
    "2x mEosNb": ASC_Nb2x_points_3pops,
    "3x mEosNb": ASC_Nb3x_points_3pops,
    "4x mEosNb": ASC_Nb3x_points_3pops,
    "5x mEosNb": ASC_Nb5x_points_3pops, #this only has 2 pops
    "6x mEosNb": ASC_Nb5x_points_3pops   #this only has 2 pops
}

In [ ]:
sesA_points = {
    1: [
    [3, 0.5],
    [10, 0.25],
    [2000, 0.25],
    [2000, 1.5],
    [3, 1.5]
    ]
}

sesA_perPlotBoundaries = {
    "Control": sesA_points,
    "1x mEosNb": sesA_points,
    "2x mEosNb": sesA_points,
    "3x mEosNb": sesA_points,
    "4x mEosNb": sesA_points,
    "5x mEosNb": sesA_points,
    "6x mEosNb": sesA_points   
}

In [ ]:
ASCUngatedData = []
for title, files in ASCUngatedFiles:
    data = NanobodyPaperPlotting.readDataFromFilelist(files)
    data, _ = NanobodyPaperPlotting.gateBDFP(data, title)
    # data = NanobodyPaperPlotting.labelFromPathPoints(data, mEos1x_perPlotBoundaries[title])
    ASCUngatedData.append((title, data))

ASCRegated3PopsData = []
for title, files in ASCUngatedFiles:
    data = NanobodyPaperPlotting.readDataFromFilelist(files)
    data, _ = NanobodyPaperPlotting.gateBDFP(data, title)
    data = NanobodyPaperPlotting.labelFromPathPoints(data, ASC_3pops_perPlotBoundaries[title])
    ASCRegated3PopsData.append((title, data))

ASCRegated2PopsData = []
for title, files in ASCUngatedFiles:
    data = NanobodyPaperPlotting.readDataFromFilelist(files)
    data, _ = NanobodyPaperPlotting.gateBDFP(data, title)
    data = NanobodyPaperPlotting.labelFromPathPoints(data, ASC_2pops_perPlotBoundaries[title])
    ASCRegated2PopsData.append((title, data))

sesAData = []
for title, files in sesAUngatedFiles:
    data = NanobodyPaperPlotting.readDataFromFilelist(files)
    data, _ = NanobodyPaperPlotting.gateBDFP(data, title)
    data = NanobodyPaperPlotting.labelFromPathPoints(data, sesA_perPlotBoundaries[title])
    sesAData.append((title, data))

In [ ]:
numCols = 7
numRows = 1


DAmFRETFig_ungated = plt.Figure(figsize, dpi=300)
DAmFRETAxs_ungated = DAmFRETFig_ungated.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.DAmFRETRowFromData(ASCUngatedData, DAmFRETAxs_ungated, addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, mpl_dpi=mpl_dpi)

reformatAxs1D(DAmFRETAxs_ungated)
DAmFRETFig_ungated.tight_layout(pad=0.25)
plt.show()
# DAmFRETFig_ungated.savefig("plots/ASC_DAmFRET.png", bbox_inches="tight")
DAmFRETFig_ungated

In [ ]:
# # numCols = 7
# # numRows = 1

# populationFig = plt.Figure(figsize, dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)

# reformatAxs1D(populationAxs)
# populationFig.tight_layout(pad=0.25)
# plt.show()
# # populationFig.savefig("plots/ASC_populations_3.png", bbox_inches="tight")
# populationFig

In [ ]:
# # squarePlotSize = 3
# # numCols = 7
# # numRows = 1

# #TODO: consider changing expressionSlice to (20,40)
# #The narrower range is to account for the different distributions of essentially unfiltered x3912b and 1x mEosNb bdfp-

# populationFig = plt.Figure(figsize, dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# # NanobodyPaperPlotting.populationRowFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, expressionSlice=(10,40), labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# NanobodyPaperPlotting.populationRowFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, expressionSlice=(20,40), labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# reformatAxs1D(populationAxs)
# populationFig.tight_layout(pad=0.25)
# plt.show()
# # populationFig.savefig("plots/ASC_binnedPopulations_3.png", bbox_inches="tight")
# populationFig

In [ ]:
#contour with no slice is not interesting

# # squarePlotSize = 3
# # numCols = 7
# # numRows = 1

# populationFig = plt.Figure(figsize, dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowWithDensityContourFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# reformatAxs1D(populationAxs)
# populationFig.tight_layout(pad=0.25)
# plt.show()
# # populationFig.savefig("plots/ASC_populationsContour_3.png", bbox_inches="tight")
# populationFig

In [ ]:
# #TESTING!!!

# populationFig = plt.Figure(figsize, dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# # NanobodyPaperPlotting.populationRowWithDensityContourFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, ylims=AmFRETLims, firstColLabelsOnly=True)
# # NanobodyPaperPlotting.populationRowFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, expressionSlice=(10,40), labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# NanobodyPaperPlotting.populationRowFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, expressionSlice=(20,40), labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# reformatAxs1D(populationAxs)
# populationFig.tight_layout(pad=0.25)
# plt.show()
# populationFig

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

populationFig = plt.Figure(figsize, dpi=300)
populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowWithDensityContourFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, ylims=AmFRETLims, firstColLabelsOnly=True)
NanobodyPaperPlotting.populationRowWithDensityContourFromData(ASCRegated3PopsData, populationAxs, "manualLabel", addTitles=False, ylims=AmFRETLims, firstColLabelsOnly=True, expressionSlice=(20,40), labelOrder=[1,2,0], mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize, textX=textX, textYMax=textYMax)
reformatAxs1D(populationAxs)
populationFig.tight_layout(pad=0.25)
plt.show()
# populationFig.savefig("plots/ASC_binnedPopulationsContour_3.png", bbox_inches="tight")
populationFig

In [ ]:
#save test

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# phaseFig = plt.Figure(figsize, dpi=300)
# phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated3PopsData, phaseAxs, "manualLabel", hueOrder=[0,2,1], addTitles=False, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)
# phaseFig.tight_layout()
# plt.show()
# # phaseFig.savefig("plots/ASC_phase_3.png")
# phaseFig

In [ ]:
squarePlotSize = 3
numCols = 7
numRows = 1

#s is arm thickness, linewidths is arm length. I think they are drawing the lines in the short direction.
plusS = 0.5
plusLinewidths = 3.5

phaseFig = plt.Figure(figsize, dpi=300)
phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated3PopsData, phaseAxs, "manualLabel", hueOrder=[0,2,1], addTitles=False, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)

for i, (title, data) in enumerate(ASCRegated3PopsData):
    for label in data["manualLabel"].unique():
        labelData = data[data["manualLabel"] == label]
        # phaseAxs[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal[label], s=12, alpha=0.75, linewidths=0)
        phaseAxs[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal_darker[label], alpha=1, marker="+", s=plusS, linewidths=plusLinewidths)

reformatAxs1D(phaseAxs)
phaseFig.tight_layout(pad=0.25)
plt.show()
# phaseFig.savefig("plots/ASC_phase_3_medianPoints.png", bbox_inches="tight")
phaseFig

In [ ]:
# #testing control plots

# squarePlotSize = 3
# numCols = 7
# numRows = 1

# #s is arm thickness, linewidths is arm length. I think they are drawing the lines in the short direction.
# plusS = 0.5
# plusLinewidths = 3.5

# phaseFig = plt.Figure(figsize, dpi=300)
# phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated3PopsData[:1], phaseAxs, "manualLabel", hueOrder=[0,2,1], addTitles=False, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)

# for i, (title, data) in enumerate(ASCRegated3PopsData):
#     for label in data["manualLabel"].unique():
#         labelData = data[data["manualLabel"] == label]
#         # phaseAxs[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal[label], s=12, alpha=0.75, linewidths=0)
#         phaseAxs[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal_darker[label], alpha=1, marker="+", s=plusS, linewidths=plusLinewidths)

# reformatAxs1D(phaseAxs)
# phaseFig.tight_layout(pad=0.25)
# plt.show()
# phaseFig

In [ ]:
# #for testing - only plot contours for first two wells

# # squarePlotSize = 3
# # numCols = 7
# # numRows = 1

# phaseFig = plt.Figure(figsize, dpi=300)
# phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated3PopsData[:2], phaseAxs, "manualLabel", hueOrder=[0,2,1], addTitles=False, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)

# for i, (title, data) in enumerate(ASCRegated3PopsData):
#     for label in data["manualLabel"].unique():
#         labelData = data[data["manualLabel"] == label]
#         # phaseAxs[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal[label], s=12, alpha=0.75, linewidths=0)
#         phaseAxs[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal_darker[label], alpha=1, marker="+", s=plusS, linewidths=plusLinewidths)

# phaseFig.tight_layout()
# plt.show()
# phaseFig

In [ ]:
# #this was just a plotting test

# squarePlotSize = 3
# numCols = 7
# numRows = 1

# phaseFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
# phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated3PopsData[:2], phaseAxs, "manualLabel", hueOrder=[0,2,1], addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)
# phaseFig.tight_layout()
# plt.show()
# phaseFig

In [ ]:
# #this was just a plotting test


# squarePlotSize = 3
# numCols = 7
# numRows = 1

# phaseFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
# phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})

# testDatas = [ASCRegated3PopsData[0], ASCRegated3PopsData[1], ASCRegated3PopsData[-3]] #control won't be plotted anyway

# # NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated3PopsData[0:1], phaseAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=np.linspace(0.1, 2, 9))
# # NanobodyPaperPlotting.BDFPAcceptorRowFromData(testDatas, phaseAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, alpha=None, linewidths=np.linspace(0.5, 2, 9)) #thickest lines seemed too thick
# # NanobodyPaperPlotting.BDFPAcceptorRowFromData(testDatas, phaseAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, alpha=None, linewidths=np.linspace(0.5, 1.5, 9)) #thickest lines still seemed too thick
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(testDatas, phaseAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, alpha=None, linewidths=np.linspace(0.5, 1.25, 9))

# phaseFig.tight_layout()
# plt.show()
# phaseFig


In [ ]:
#NOTE: the following blocks were commented out because 2 population ASC was less interesting than 3 population ASC

# squarePlotSize = 3
# numCols = 7
# numRows = 1

# populationFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowFromData(ASCRegated2PopsData, populationAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, ylims=AmFRETLims, firstColLabelsOnly=True)
# populationFig.tight_layout()
# plt.show()
# # populationFig.savefig("plots/ASC_populations_2.png")
# populationFig

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# populationFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowWithDensityContourFromData(ASCRegated2PopsData, populationAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, ylims=AmFRETLims, firstColLabelsOnly=True)
# populationFig.tight_layout()
# plt.show()
# # populationFig.savefig("plots/ASC_populationsContour_2.png")
# populationFig

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# populationFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
# populationAxs = populationFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowWithDensityContourFromData(ASCRegated2PopsData, populationAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, ylims=AmFRETLims, firstColLabelsOnly=True, expressionSlice=(10,40), labelOrder=[2,1,0])
# populationFig.tight_layout()
# plt.show()
# # populationFig.savefig("plots/ASC_binnedPopulationsContour_2.png")
# populationFig

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# phaseFig = plt.Figure((squarePlotSize*numCols ,squarePlotSize*numRows), dpi=300)
# phaseAxs = phaseFig.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(ASCRegated2PopsData, phaseAxs, "manualLabel", addTitles=True, rowTitle="ASC", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True)
# phaseFig.tight_layout()
# plt.show()
# # phaseFig.savefig("plots/ASC_phase_2.png")
# phaseFig

In [ ]:
squarePlotSize = 3
numCols = 7
numRows = 1

DAmFRETFig_sesA = plt.Figure(figsize, dpi=300)
DAmFRETAxs_sesA = DAmFRETFig_sesA.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.DAmFRETRowFromData(sesAData, DAmFRETAxs_sesA, addTitles=False, ylims=AmFRETLims_sesA, firstColLabelsOnly=True, mpl_dpi=mpl_dpi)
reformatAxs1D(DAmFRETAxs_sesA)
DAmFRETFig_sesA.tight_layout(pad=0.25)
plt.show()
# DAmFRETFig_sesA.savefig("plots/sesA_DAmFRET.png", bbox_inches="tight")
DAmFRETFig_sesA

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# populationFig_sesA = plt.Figure(figsize, dpi=300)
# populationAxs_sesA = populationFig_sesA.subplots(numRows, numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowFromData(sesAData, populationAxs_sesA, "manualLabel", addTitles=False, ylims=AmFRETLims_sesA, firstColLabelsOnly=True, mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# reformatAxs1D(populationAxs_sesA)
# populationFig_sesA.tight_layout(pad=0.25)
# plt.show()
# # populationFig_sesA.savefig("plots/sesA_populations.png", bbox_inches="tight")
# populationFig_sesA

In [ ]:
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# populationFig_sesA = plt.Figure(figsize, dpi=300)
# populationAxs_sesA = populationFig_sesA.subplots(numRows, numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowFromData(sesAData, populationAxs_sesA, "manualLabel", addTitles=False, ylims=AmFRETLims_sesA, firstColLabelsOnly=True, labelOrder=[1,0], expressionSlice=(50, 200), mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# reformatAxs1D(populationAxs_sesA)
# populationFig_sesA.tight_layout(pad=0.25)
# plt.show()
# # populationFig_sesA.savefig("plots/sesA_binnedPopulations.png", bbox_inches="tight")
# populationFig_sesA

In [ ]:
#contour with no slice is not interesting
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# populationFig_sesA = plt.Figure(figsize, dpi=300)
# populationAxs_sesA = populationFig_sesA.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.populationRowWithDensityContourFromData(sesAData, populationAxs_sesA, "manualLabel", addTitles=False, ylims=AmFRETLims_sesA, firstColLabelsOnly=True, mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize)
# reformatAxs1D(populationAxs_sesA)
# populationFig_sesA.tight_layout(pad=0.25)
# plt.show()
# # populationFig_sesA.savefig("plots/sesA_populationsContour.png", bbox_inches="tight")
# populationFig_sesA

In [ ]:
squarePlotSize = 3
numCols = 7
numRows = 1

populationFig_sesA = plt.Figure(figsize, dpi=300)
populationAxs_sesA = populationFig_sesA.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.populationRowWithDensityContourFromData(sesAData, populationAxs_sesA, "manualLabel", addTitles=False, ylims=AmFRETLims_sesA, firstColLabelsOnly=True, labelOrder=[1,0], expressionSlice=(50, 200), mpl_dpi=mpl_dpi, summaryFontSize=tickFontSize, textX=textX, textYMax=textYMax)
reformatAxs1D(populationAxs_sesA)
populationFig_sesA.tight_layout(pad=0.25)
plt.show()
# populationFig_sesA.savefig("plots/sesA_binnedPopulationsContour.png", bbox_inches="tight")
populationFig_sesA

In [ ]:
#phase without crosshairs is not interesting
# squarePlotSize = 3
# numCols = 7
# numRows = 1

# phaseFig_sesA = plt.Figure(figsize, dpi=300)
# phaseAxs_sesA = phaseFig_sesA.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
# NanobodyPaperPlotting.BDFPAcceptorRowFromData(sesAData, phaseAxs_sesA, "manualLabel", hueOrder=[0,1], addTitles=True, rowTitle="SesA", rowTitleXShift=-0.03, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)
# reformatAxs1D(phaseAxs_sesA)
# phaseFig_sesA.tight_layout(pad=0.25)
# plt.show()
# # phaseFig_sesA.savefig("plots/sesA_phase.png", bbox_inches="tight")
# phaseFig_sesA

In [ ]:
squarePlotSize = 3
numCols = 7
numRows = 1

phaseFig_sesA = plt.Figure(figsize, dpi=300)
phaseAxs_sesA = phaseFig_sesA.subplots(numRows,numCols, subplot_kw={'projection': "scatter_density"})
NanobodyPaperPlotting.BDFPAcceptorRowFromData(sesAData, phaseAxs_sesA, "manualLabel", hueOrder=[0,1], addTitles=False, firstColLabelsOnly=True, xlims=expressionLims_kde, ylims=BDFPPositiveBDFPSSCLims_kde, emptyFirstCol=True, linewidths=None)

for i, (title, data) in enumerate(sesAData):
    for label in data["manualLabel"].unique():
        labelData = data[data["manualLabel"] == label]
        # phaseAxs_sesA[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal[label], s=12, alpha=0.75, linewidths=0)
        phaseAxs_sesA[i].scatter(labelData["Acceptor/SSC"].median(), labelData["BDFP/SSC"].median(), color=colors_decimal_darker[label], alpha=1, marker="+", s=plusS, linewidths=plusLinewidths)

reformatAxs1D(phaseAxs_sesA)
phaseFig_sesA.tight_layout(pad=0.25)
plt.show()
# phaseFig_sesA.savefig("plots/sesA_phase_medianPoints.png", bbox_inches="tight")
phaseFig_sesA

### proportion high-AmFRET plot

In [ ]:
#these files were from a different run, so the concentration vs AmFRET values were slighly different (to be expected due to experimental variation)

ASC_standardPoints_2pops_triplicates = {
    1: [
    [0.9, 0.35],
    [10, 0.2],
    [40, 0.17],
    [2000, 0.17],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

# ASC_2pops_perPlotBoundaries_triplicates = {
#     "Control": ASC_standardPoints_2pops_triplicates,
#     "1x mEosNb": ASC_standardPoints_2pops_triplicates,
#     "2x mEosNb": ASC_standardPoints_2pops_triplicates,
#     "3x mEosNb": ASC_standardPoints_2pops_triplicates,
#     "4x mEosNb": ASC_standardPoints_2pops_triplicates,
#     "5x mEosNb": ASC_standardPoints_2pops_triplicates,
#     "6x mEosNb": ASC_standardPoints_2pops_triplicates   
# }

ASC_Nb1x_points_3pops_triplicates = {
    1: ASC_standardPoints_2pops_triplicates[1],

    2: [
    # [3, 0.72],
    # [2000, 0.72],
    [0.9, 0.75],
    [2000, 0.75],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

ASC_Nb2x_points_3pops_triplicates = {
    1: ASC_standardPoints_2pops_triplicates[1],

    2: [
    [0.9, 0.88],
    [2000, 0.88],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

ASC_Nb3x_points_3pops_triplicates = {
    1: ASC_standardPoints_2pops_triplicates[1],

    2: [
    [0.9, 0.96],
    [2000, 0.96],
    [2000, 1.5],
    [0.9, 1.5],
    ]
}

ASC_3pops_perPlotBoundaries_triplicates = {
    "Control": ASC_standardPoints_2pops_triplicates,  #this only has 2 pops
    "1x mEosNb": ASC_Nb1x_points_3pops_triplicates,
    "2x mEosNb": ASC_Nb2x_points_3pops_triplicates,
    "3x mEosNb": ASC_Nb3x_points_3pops_triplicates,
    "4x mEosNb": ASC_Nb3x_points_3pops_triplicates,
    "5x mEosNb": ASC_standardPoints_2pops_triplicates, #this only has 2 pops
    "6x mEosNb": ASC_standardPoints_2pops_triplicates   #this only has 2 pops
}

sesA_points_triplicates = {
    1: [
    [3, 0.5],
    [10, 0.25],
    [2000, 0.25],
    [2000, 1.5],
    [3, 1.5]
    ]
}

In [ ]:
ASCUngatedFiles_triplicates = {
    1: ['data/ASC/complete/mating_plate1_rep1/B5.fcs',
        'data/ASC/complete/mating_plate1_rep2/B5.fcs',
        'data/ASC/complete/mating_plate1_rep3/B5.fcs'],
    2: ['data/ASC/complete/mating_plate1_rep1/C5.fcs',
        'data/ASC/complete/mating_plate1_rep2/C5.fcs',
        'data/ASC/complete/mating_plate1_rep3/C5.fcs'],
    3: ['data/ASC/complete/mating_plate1_rep1/D5.fcs',
        'data/ASC/complete/mating_plate1_rep2/D5.fcs',
        'data/ASC/complete/mating_plate1_rep3/D5.fcs'],
    4: ['data/ASC/complete/mating_plate1_rep1/E5.fcs',  
        'data/ASC/complete/mating_plate1_rep2/E5.fcs',
        'data/ASC/complete/mating_plate1_rep3/E5.fcs'],
    5: ['data/ASC/complete/mating_plate1_rep1/F5.fcs',
        'data/ASC/complete/mating_plate1_rep2/F5.fcs',
        'data/ASC/complete/mating_plate1_rep3/F5.fcs'],
    6: ['data/ASC/complete/mating_plate1_rep1/G5.fcs',
        'data/ASC/complete/mating_plate1_rep2/G5.fcs',
        'data/ASC/complete/mating_plate1_rep3/G5.fcs']
}

sesAUngatedFiles_triplicates = {
    1: ['data/SesA/mating_plate1_rep1/B6.fcs',
        'data/SesA/mating_plate1_rep2/B6.fcs',
        'data/SesA/mating_plate1_rep3/B6.fcs'],
    2: ['data/SesA/mating_plate1_rep1/C6.fcs',
        'data/SesA/mating_plate1_rep2/C6.fcs',
        'data/SesA/mating_plate1_rep3/C6.fcs'],
    3: ['data/SesA/mating_plate1_rep1/D6.fcs',
        'data/SesA/mating_plate1_rep2/D6.fcs',
        'data/SesA/mating_plate1_rep3/D6.fcs'],
    4: ['data/SesA/mating_plate1_rep1/E6.fcs',
        'data/SesA/mating_plate1_rep2/E6.fcs',
        'data/SesA/mating_plate1_rep3/E6.fcs'],
    5: ['data/SesA/mating_plate1_rep1/F6.fcs',
        'data/SesA/mating_plate1_rep2/F6.fcs',
        'data/SesA/mating_plate1_rep3/F6.fcs'],
    6: ['data/SesA/mating_plate1_rep1/G6.fcs',
        'data/SesA/mating_plate1_rep2/G6.fcs',
        'data/SesA/mating_plate1_rep3/G6.fcs']
}

In [ ]:
ASCDataDict = defaultdict(list) #{1: [data, data, data], ...}

for mEosNbCount, fileList in ASCUngatedFiles_triplicates.items(): 
    for file in fileList:
        data = NanobodyPaperPlotting.readDataFromFilelist([file])
        title = f"{mEosNbCount}x mEosNb, {file.split('/')[-2].split('_')[-1]}"
        
        data = data[data["Acceptor/SSC"] > 0]
        data = data[(data["Acceptor/SSC"] >= expressionLims[0]) & (data["Acceptor/SSC"] <= expressionLims[1])]
        dataHighBDFP, _ = NanobodyPaperPlotting.gateBDFP(data, title)

        if mEosNbCount == 1:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, ASC_Nb1x_points_3pops_triplicates)
        
            #also get bdfp- control
            dataLowBDFP, _ = NanobodyPaperPlotting.gateBDFP(data, "Control") #lie to force bdfp-
            NanobodyPaperPlotting.labelFromPathPoints(dataLowBDFP, ASC_standardPoints_2pops_triplicates)
            
        elif mEosNbCount == 2:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, ASC_Nb2x_points_3pops_triplicates)
        elif mEosNbCount == 3:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, ASC_Nb3x_points_3pops_triplicates)
        elif mEosNbCount == 4:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, ASC_Nb3x_points_3pops_triplicates)
        elif mEosNbCount == 5:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, ASC_standardPoints_2pops_triplicates)
        elif mEosNbCount == 6:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, ASC_standardPoints_2pops_triplicates)

        ASCDataDict[mEosNbCount].append(dataHighBDFP)
        if mEosNbCount == 1:
            ASCDataDict[0].append(dataLowBDFP)

            NanobodyPaperPlotting.plotDAmFRETClusters(dataLowBDFP["Acceptor/SSC"], dataLowBDFP["AmFRET"], dataLowBDFP["manualLabel"], logX=True, title=title+" bdfp-", colors=colors_decimal)
            
        # NanobodyPaperPlotting.plotDAmFRETDensity(data["Acceptor/SSC"], data["AmFRET"], logX=True, title=title)
        NanobodyPaperPlotting.plotDAmFRETClusters(dataHighBDFP["Acceptor/SSC"], dataHighBDFP["AmFRET"], dataHighBDFP["manualLabel"], logX=True, title=title, colors=colors_decimal)
        plt.show()

In [ ]:
sesADataDict = defaultdict(list) #{1: [data, data, data], ...}

for mEosNbCount, fileList in sesAUngatedFiles_triplicates.items(): 
    for file in fileList:
        data = NanobodyPaperPlotting.readDataFromFilelist([file])
        title = f"{mEosNbCount}x mEosNb, {file.split('/')[-2].split('_')[-1]}"
        
        data = data[data["Acceptor/SSC"] > 0]
        data = data[(data["Acceptor/SSC"] >= expressionLims[0]) & (data["Acceptor/SSC"] <= expressionLims[1])]
        dataHighBDFP, _ = NanobodyPaperPlotting.gateBDFP(data, title)

        if mEosNbCount == 1:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, sesA_points_triplicates)
            dataLowBDFP, _ = NanobodyPaperPlotting.gateBDFP(data, "Control") #lie to force bdfp-
            NanobodyPaperPlotting.labelFromPathPoints(dataLowBDFP, sesA_points_triplicates)

        elif mEosNbCount == 2:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, sesA_points_triplicates)
        elif mEosNbCount == 3:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, sesA_points_triplicates)
        elif mEosNbCount == 4:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, sesA_points_triplicates)
        elif mEosNbCount == 5:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, sesA_points_triplicates)
        elif mEosNbCount == 6:
            NanobodyPaperPlotting.labelFromPathPoints(dataHighBDFP, sesA_points_triplicates)

        sesADataDict[mEosNbCount].append(dataHighBDFP)
        if mEosNbCount == 1:
            sesADataDict[0].append(dataLowBDFP)

            NanobodyPaperPlotting.plotDAmFRETClusters(dataLowBDFP["Acceptor/SSC"], dataLowBDFP["AmFRET"], dataLowBDFP["manualLabel"], logX=True, title=title+" bdfp-", colors=colors_decimal)
      
        
        # NanobodyPaperPlotting.plotDAmFRETDensity(data["Acceptor/SSC"], data["AmFRET"], logX=True, title=title)
        NanobodyPaperPlotting.plotDAmFRETClusters(dataHighBDFP["Acceptor/SSC"], dataHighBDFP["AmFRET"], dataHighBDFP["manualLabel"], logX=True, title=title, colors=colors_decimal)
        plt.show()

In [ ]:
numDivisions = 30
boundaries = np.logspace(expressionLims_kde[0], expressionLims_kde[1], numDivisions)
minPerBin = 75

meanExpressionInBins = []
for i, lowerBoundary in enumerate(boundaries[:-1]):
    upperBoundary = boundaries[i + 1]

    avgAccSSC = 10 ** ((np.log10(lowerBoundary) + np.log10(upperBoundary)) / 2)
    meanExpressionInBins.append(avgAccSSC)

In [ ]:
pop1Summaries_ASC = [] #list of list of floats for EACH well [[[1xmEosNb_rep1Floats], [1xmEosNb_rep2], ...], ...]

for i, (mEosNbCount, dataList) in enumerate(ASCDataDict.items()):
    pop1Summaries_ASC.append([])
    
    for data in dataList:
        pop1TempSummary = []
        
        for i, lowerBoundary in enumerate(boundaries[:-1]):
            upperBoundary = boundaries[i + 1]
        
            avgAccSSC = (lowerBoundary + upperBoundary) / 2
        
            dataInSlice = data[(data["Acceptor/SSC"] >= lowerBoundary) & (data["Acceptor/SSC"] <= upperBoundary)]
            labels, labelCounts = np.unique(dataInSlice["manualLabel"], return_counts=True)
            labelCountDict = dict(zip(labels, labelCounts))
    
            if sum(labelCounts) < minPerBin: #don't trust any values because there isn't enough data
                pop1TempSummary.append(None)
            
            else:  
                defaultValue = 0 #if a label doesn't appear, there are no cells with that label

                #MAJOR CAUTION: this version of ASC gating separates both high fret pops, so treat both 1 and 2 as high fret!
                pop1TempSummary.append((labelCountDict.get(1, defaultValue) / sum(labelCounts)) + (labelCountDict.get(2, defaultValue) / sum(labelCounts)))
                
                
        pop1Summaries_ASC[-1].append(pop1TempSummary)

pop1SummariesArray_ASC = np.array(pop1Summaries_ASC, dtype=float)

In [ ]:
pop1Summaries_sesA = [] #list of list of floats for EACH well [[[1xmEosNb_rep1Floats], [1xmEosNb_rep2], ...], ...]

for i, (mEosNbCount, dataList) in enumerate(sesADataDict.items()):
    pop1Summaries_sesA.append([])

    #handle ignored replicates (just 1x mEosNb rep2)
    if len(dataList) == 2:
        pop1Summaries_sesA[-1].append([None] * (numDivisions - 1))
    
    
    for data in dataList:
        pop1TempSummary = []
        
        for i, lowerBoundary in enumerate(boundaries[:-1]):
            upperBoundary = boundaries[i + 1]
        
            avgAccSSC = (lowerBoundary + upperBoundary) / 2
        
            dataInSlice = data[(data["Acceptor/SSC"] >= lowerBoundary) & (data["Acceptor/SSC"] <= upperBoundary)]
            labels, labelCounts = np.unique(dataInSlice["manualLabel"], return_counts=True)
            labelCountDict = dict(zip(labels, labelCounts))
    
            if sum(labelCounts) < minPerBin: #don't trust any values because there isn't enough data
                pop1TempSummary.append(None)
            
            else:  
                defaultValue = 0 #if a label doesn't appear, there are no cells with that label

                pop1TempSummary.append(labelCountDict.get(1, defaultValue) / sum(labelCounts))
            
                
        pop1Summaries_sesA[-1].append(pop1TempSummary)
pop1SummariesArray_sesA = np.array(pop1Summaries_sesA, dtype=float)

In [ ]:
def reformatFposAxs(axs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        # for i, row in enumerate(axs):
        for j, ax in enumerate(axs):
            ax.xaxis.set_label_coords(0.5, -0.15)
            ax.yaxis.set_label_coords(-0.15, 0.5)
    
            ax.tick_params(axis='both', which='major', pad=1, width=0.5, length=2.5) #default length is 3.5
            # ax.minorticks_off()
            ax.tick_params(axis='x', which='minor', width=0.5, length=1.5)           #default length is 2
            ax.xaxis.set_ticklabels(ax.xaxis.get_ticklabels(), fontsize=tickFontSize)
            ax.yaxis.set_ticklabels(ax.yaxis.get_ticklabels(), fontsize=tickFontSize)
            
            for spine in ax.spines.values():
                spine.set_linewidth(0.5)

In [ ]:
fontSize = 6
tickFontSize = 4
plt.rcParams.update({'font.size': fontSize})

combinedFig = plt.Figure((3.25, 1.5), dpi=300)
combinedAxs = combinedFig.subplots(1,2)

# makeFPosPlot(meanExpressionInBins, pop1Summaries_ASC, combinedAxs[0], [title for title, files in ASCFiles])
# makeFPosPlot(meanExpressionInBins, pop1Summaries_sesA, combinedAxs[1], [title for title, files in sesAUngatedFiles])

labels = ["Control",
          "1x mEosNb",
         "2x mEosNb",
         "3x mEosNb",
         "4x mEosNb",
         "5x mEosNb",
         "6x mEosNb"
         ]

for i, pop1SummariesGroup in enumerate(pop1SummariesArray_ASC):
    combinedAxs[0].errorbar(boundaries[:-1], np.nanmean(pop1SummariesGroup, axis=0), yerr=np.nanstd(pop1SummariesGroup, axis=0), capsize=1, linewidth=0.5, capthick=0.5, label=i)
# combinedAxs[0].legend(labels=[0,1,2,3,4,5,6])
combinedAxs[0].legend(labels=labels)
combinedAxs[0].set_xscale("log")

for i, pop1SummariesGroup in enumerate(pop1SummariesArray_sesA):
    combinedAxs[1].errorbar(boundaries[:-1], np.nanmean(pop1SummariesGroup, axis=0), yerr=np.nanstd(pop1SummariesGroup, axis=0), capsize=1, linewidth=0.5, capthick=0.5, label=i)
# combinedAxs[1].legend(labels=[0,1,2,3,4,5,6])
combinedAxs[1].legend(labels=labels)
combinedAxs[1].set_xscale("log")

#modified from https://stackoverflow.com/a/15551976 to remove errorbars from the legend
# get handles
handles, labels_mpl = combinedAxs[0].get_legend_handles_labels()
# remove the errorbars
handles = [h[0] for h in handles]
# use them in the legend
combinedAxs[0].legend(handles, labels, fontsize=tickFontSize)

# get handles
handles, labels_mpl = combinedAxs[1].get_legend_handles_labels()
# remove the errorbars
handles = [h[0] for h in handles]
# use them in the legend
combinedAxs[1].legend(handles, labels, fontsize=tickFontSize)

combinedAxs[0].set_xlabel("mEos3 conc. (p.d.u.)")
combinedAxs[1].set_xlabel("mEos3 conc. (p.d.u.)")

# combinedAxs[0].set_ylabel("Proportion nucleated")
# combinedAxs[1].set_ylabel("Proportion nucleated")
combinedAxs[0].set_ylabel("Fraction high-AmFRET")
combinedAxs[1].set_ylabel("Fraction high-AmFRET")

reformatFposAxs(combinedAxs)
combinedFig.tight_layout(w_pad=2)
# combinedFig.savefig("plots/ASC_SesA_fpos.png")
combinedFig